In [1]:
# Run this cell to load the necessary helper functions
#
#

import sys
b= sys.path
sys.path=['/home/jovyan/analysis'] + b
import osiris
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from osh5vis import new_fig, osplot
from osh5io import *
import osh5io
import osh5vis
from osh5utils import fft
import glob
from ipywidgets import interact_manual,fixed,Layout,interact, FloatSlider
import ipywidgets as widgets
interact_calc=interact_manual.options(manual_name="Run Leap Frog Program")
import os
from osiris import tajima
from h5_utilities import *
from analysis import *
import matplotlib.colors as colors

## Leap Frog Integration Method:

#####  Ver 1.0 (FST for HED Summer School 2019)
    
    An important numerical technique used in particle-in-cell (PIC) method is the leap-frog integration method.  This notebook aims to demonstrate the properties of the method, and by extension helps you make better choices in designing your simulations.  

The stability of an integration method can be obtained by solving the so-called "characteristic polynomial" associated with the numerical method.  The most famous example of this is the Fibonnacci series, where the series is defined as follows:

$x^n = x^{n-1} + x^{n-2}$

$x^0 = 1$,   
$x^1 = 1$

This series yields the famous "Golden ratio", 1.618, which is the asymptotic ratio between two successive numbers in the above series.  

$lim_{n \rightarrow \infty} \frac{x^{n+1}}{x^{n}} = 1.618$

The stability of a numerical method is obtained by looking at the ratio above.  If the modulus of the ratio is greater than 1, then any small error (which arises from the numerical accuracy of the method) is amplified by the method and will grow without bound.  The ratio can be calculated by assuming that asymptotically (as n approaches infinity), the series has the following functional dependence:

$lim_{n \rightarrow \infty} x^{n} = r^n$ 


### Stability Condition for the Leap Frog Scheme For the Simple Harmonic Oscillator Problem:

To perform stability condition for the leap frog method, let's consider the simple harmonic oscillator problem, i.e.:

$\frac{d x}{d t} = v = i \omega_0 x $

$\frac{d v}{dt} = - \omega_0^2 x $

combine that with the leap frog scheme:

$x^{n+1/2} = x^{n-1/2} + dt \ ( v^{n}$ )

$ v^{n+1} = v^{n} + dt \ (- \omega_0^2 x^{n+1/1})$

(In the code I use the normalization of $\omega_0 \equiv 1$)

To evaluate the stability condition for the Leap Frog scheme for this problem, let's look at the equation:

$x^{n+1/2} = x^{n-1/2} + dt i \omega_0 x^{n}$

We can convert that to a polynomial, by assuming that $lim_{n \rightarrow \infty} x^{n} = r^n$, then we get a 
second order polynomial for the ratio $r$.

$r^2  - (i \Delta t \omega_0) r -1 = 0$

and the roots are:

$r = \frac{1}{2} \left(  (i \Delta t \omega_0) \pm \sqrt{ -\omega^2 \Delta t^2 + 4 } \right)$

For $\omega_0 \Delta t $ less than 2, the modulus of the complex number $r$ is 1.  Therefore,  
the leap-frog integration method is stable for 

$\omega_0 \Delta t < 2.0$

for $\omega_0 \Delta t > 2$ the modulus for $r$ is greater than 1 and the numerical scheme will generate solutions that are not oscillatory, and produce exponential growing solutions.  This notebook will demonstrate this by solving the simple harmonic oscillator with leap frog using Python.  


You can also use this notebook to explore other integration methods.




In [2]:
# here we define the acceleration function for the simple harmonic oscillator
#
# for simplicity sake we will set omega0 to 1 in this notebook.  Everything is normalized w.r.t. omega0 here
#
#
tbegin = 0.0
tend=100.0
tplot = 1.0


def force(x,omega0):
    return(-omega0*omega0*x)

#
#
#
def leap_frog_engine(x0=1,v0=0,dt=0.2,tend=100):
# here is the leap-frog engine
# you can modify this or try other types of integrator
#
    print(x0)
    print(v0)
    print(dt)
    print(tend)
    nt = 1
    nt=int(tend/dt)
    print(nt)
    x_array=np.zeros(nt)
    v_array=np.zeros(nt)
    x_theory=np.zeros(nt)
    v_theory=np.zeros(nt)
    
    i=0
    time=0.0
    # x_array[0]=x0-v0*dt/2.0
    x_array[0]=x0*np.cos(time-dt/2.0)+v0*np.sin(time-dt*0.5)
    v_array[0]=v0
    x_theory[0]=x0*np.cos(time-dt/2.0)+v0*np.sin(time-dt*0.5)
    v_theory[0]=v0*np.cos(time)-x0*np.sin(time)
    t_array=np.arange(nt)*dt
    t_half_array=np.arange(nt)*dt - dt*0.5
    i=1
    while i<nt:
        time=i*dt
        x_array[i]=x_array[i-1] + dt*v_array[i-1]
        v_array[i]=v_array[i-1] - dt*x_array[i]
        # x_array[i]=(x_array[i-1] + np.sin(dt)*v_array[i-1])
        # v_array[i]=(v_array[i-1] - np.sin(dt)*x_array[i])
        x_theory[i]=x0*np.cos(time-dt*0.5)+v0*np.sin(time-dt*0.5)
        #x_theory[i]=x0*np.cos(time)+v0*np.sin(time)
        v_theory[i]=v0*np.cos(time)-x0*np.sin(time)
        i+=1
    plt.figure(figsize=(12,12))
    plt.subplot(221)
    plt.scatter(x_array,v_array,marker='^',label='Leap Frog')
    plt.scatter(x_theory,v_theory,marker='o',label='Theory')
    plt.xlabel('position')
    plt.ylabel('velocity')
    plt.xlim([-3,3])
    plt.ylim([-3,3])
    plt.legend()
    plt.subplot(222)
    plt.scatter(t_half_array,x_array)
    plt.scatter(t_half_array,x_theory)
    plt.xlabel('Time')
    plt.ylabel('position(t)')
    plt.subplot(223)
    plt.scatter(t_array,v_array)
    plt.scatter(t_array,v_theory)
    plt.xlabel('Time')
    plt.ylabel('velocity(t)')
    plt.subplot(224)
    plt.plot(t_array,v_array-v_theory)
    plt.show()
    
def leap_frog_widget():
    
    style = {'description_width': '350px'}
    layout = Layout(width='55%')

  
    a = widgets.FloatSlider(value=1.000, min=0.0, max=2.0,step=0.1, description='x0:',
                            orientation='horizontal',readout=True,readout_format='.1f')
    b = widgets.FloatSlider(value=0.000, min=0.0, max=2.0,step=0.1, description='v0/omega_0:',
                            orientation='horizontal',readout=True,readout_format='.1f')
    
    c = widgets.FloatText(value=0.2,description='dt:',style=style,layout=layout)
    d = widgets.FloatText(value=300.0,description='tend:',style=style,layout=layout)
    

    leap_frog = interact_calc(leap_frog_engine, x0=a,v0=b,dt=c,tend=d);
    leap_frog.widget.manual_button.layout.width='400px'
    
    
leap_frog_widget()

interactive(children=(FloatSlider(value=1.0, description='x0:', max=2.0, readout_format='.1f'), FloatSlider(va…

####  Some questions:

(1)  Can anyone explain the behavior of the solver as a function of dt, for dt < 2?  Why is there a modulation in the error?

(2)  (very advanced) For the simple harmonic problem, can a exact integration be implemented for dt < 2?

